In [35]:
import boto3
import csv
import json
import os
import requests
from io import BytesIO;
from os import listdir
from os.path import isfile, join

In [36]:
path = "/home/propietari/Documents/fitxers importants/WRI/Scraping_results/"

onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]


In [37]:
onlyfiles

['ElSalvador_20210730.csv',
 'USFR_2_20210311.csv',
 'Mexico_20210423.csv',
 'USFR_20210311.csv',
 'USFR_1_20210311.csv',
 'USFR_20210702.csv',
 'LeyChile_20210702.csv',
 'India_20210310.csv',
 'Mexico_20210703.csv',
 'USFR_20210310.csv',
 'India_20210703.csv',
 'India_20210812_2011.csv',
 'Mexico_20210809.csv',
 'El_Salvador_no_duplicates.csv']

In [38]:
# path = 'C:/Users/jordi/Documents/claus/'
key_path = '/home/propietari/Documents/claus/'
filename = 'AWS_S3_keys_wri.json'
file = key_path + filename
with open(file, 'r') as dict:
    credentials = json.load(dict)
                                      
KEY = list(credentials)[0]
SECRET = list(credentials.values())[0]
# s3BucketName = "wri-testing"
s3BucketName = "wri-nlp-policy"
# region = 'eu-central-1'
region = "us-east-1"

s3 = boto3.resource(
    service_name = 's3',
    region_name = region,
    aws_access_key_id = KEY,
    aws_secret_access_key = SECRET
)


### El Salvador

In [5]:
# Loading the csv files from scraping to be compared
file_position = 0
with open(path + onlyfiles[file_position], 'r', encoding = "Latin1") as f:
    reader = csv.reader(f)
    ElSalvador_july = [row[3] for row in reader]

file_position = 13
with open(path + onlyfiles[file_position], 'r', encoding = "Latin1") as f:
    reader = csv.reader(f)
    El_Salvador_march = [row[7] for row in reader]

In [6]:
print(f"There are {len(El_Salvador_march) - len(set(El_Salvador_march))} duplicates in the march version")
print(f"There are {len(ElSalvador_july) - len(set(ElSalvador_july))} duplicates in the august version")
print(f"There are {len(set(El_Salvador_march))} items in the March file and {len(set(ElSalvador_july))} in the August one")
print(f"There are {len(ElSalvador_july) - len(set(ElSalvador_july))} duplicates in the new version")
print(f"There are {len(set(El_Salvador_march) - set(ElSalvador_july))} files in the old version that are not in the new version")
print(f"There are {len(set(ElSalvador_july) - set(El_Salvador_march))} files in the new version that are not in the old version")
print(f"There are {len(set(El_Salvador_march) | set(ElSalvador_july))} unique ids in the union of the files")

There are 0 duplicates in the march version
There are 0 duplicates in the august version
There are 460 items in the March file and 271 in the August one
There are 0 duplicates in the new version
There are 236 files in the old version that are not in the new version
There are 47 files in the new version that are not in the old version
There are 507 unique ids in the union of the files


In [12]:

keys_dict = {key : 0 for key in list(set(El_Salvador_march) | set(ElSalvador_july))}

In [16]:
path = "/home/propietari/Documents/GitHub/policy-data-analyzer/tasks/database_analysis/input/"
file_name = "ElSalvadorKeys.json"
file = path + file_name

with open(file, 'w') as fp:
    json.dump(keys_dict, fp)

In [32]:
# Going to merge the two files in one. The fields were not saved in the same order so it is necessary to rearrange them

file_position = 0
with open(path + onlyfiles[file_position], 'r', encoding = "Latin1") as f:
    reader = csv.reader(f)
    ElSalvador_july = list(reader)

file_position = 13
with open(path + onlyfiles[file_position], 'r') as f:
    reader = csv.reader(f)
    El_Salvador_march = list(reader)
    
final_list = []
ids = {}
for item in El_Salvador_march:
    if item[7] not in ids:
        final_item = []
        final_item.append(item[7])
        final_item.append('El Salvador')
        final_item.append('Diario Oficial')
        final_item.append(item[1])
        final_item.append(item[4])
        final_item.append(item[2])
        final_item.append(item[6])
        final_item.append(item[5])
        final_item.append(item[3])
        final_list.append(final_item)
        ids[item[7]] = 0
    
for item in ElSalvador_july:
    if item[3] not in ids:
        final_item = []
        final_item.append(item[3])
        final_item.append(item[1])
        final_item.append(item[2])
        final_item.append(item[0])
        final_item.append(item[7])
        final_item.append(item[6])
        final_item.append(item[10])
        final_item.append(item[9])
        final_item.append(item[4])
        final_list.append(final_item)
        ids[item[3]] = 0

In [21]:
# Checking the differences with bucket files

language = "spanish"
in_prefix = f"{language}_documents/HSSC/updated_sentences/"
ids_S3 = {}
for i, obj in enumerate(s3.Bucket(s3BucketName).objects.all().filter(Prefix = in_prefix)):
    if ".json" in obj.key and "ing" not in obj.key:
        name = obj.key.split("/")[-1].split("_")[0]
        ids_S3[name] = 0
        

In [33]:
print(f"The number of scraped items that are not in sentences is {len(ids.keys() - ids_S3.keys())}")

The number of scraped items that are not in sentences is 236


In [34]:
lst = list(ids.keys())
print(list(ids.keys() & ids_S3.keys()))

['369c7dd6018883c0f1a1e1c42443faa29098a0e3', 'a048b3c834b4a337390f9d1cc402d7e3b1467585', '5a47e49af7354d5edeef87127849614ffe502aa3', '5ea6d0cf5496202a9811826aae1028d51d1ae8bf', 'c00e1d3a9433503e81be2a91e32ce82f1370dbd5', 'd6a7f205525cd506c062e2b7246c724846df893e', '74a6962f1010fd15da76d240d2c356dcd5c5742a', '881adebad32064d76c75bf630cc3dc4412b09feb', '081affee12e9cd3823f6e692f4b245d7b426e040', '06dafc0a507e270d9b23b192489cb1012aed1a8f', '15169f795b04dd22b4bb7176b1a33d0daae1af1a', '3610a3c2c1d7d560d928479bb782136b142d22b3', 'b8922bef7759a7b117730fe9e88dfc26c01a4cb0', 'e54692718b4806efc931278a143d5c8099c0a258', 'bf78089f6178944158517f27b9b7cd01c1979c75', '3eb7dca2b45b82baefdbc429892c7948b8de9127', '8e2d3161a289edffd045a1041f6cec2831109c77', '6458c88c621d5502a5c976e0e835e53f0cdec420', '354dd06f542354eb73989b3b9d96c34fefead639', '7d5b453cfb2554275702fb7a4c70a8c11a55f616', '7f9793ab4e41eba18a1325efb4153766c2e0971a', '3c8c7a4009323ca842923f02c56c10f6d697d801', '908e58192c424feac85907830cd5f6

In [98]:
# Here we go and fetch the missing files and put them in the raw_pdf_updated folder
prefix = f"{language}_documents/raw_pdf_updated/"


counter = 0
bucket = s3.Bucket(s3BucketName)
for item in final_list:
    if item[0] not in ids_S3:
        counter += 1
        key = prefix + item[0] + ".pdf"
        with requests.get(item[8], stream=True) as r:
            bucket.upload_fileobj(r.raw, key)
print(counter)

236


### Chile

In [4]:
file_position = 6
with open(path + onlyfiles[file_position], 'r', encoding="ISO-8859-1") as f:#, encoding = "Latin1"
    reader = csv.reader(f)
    chile = list(reader)
    
final_list = []
ids = {}
dates = {}
for item in chile:
    if item[3].split(".")[0] not in ids:
        final_item = []
        final_item.append(item[3].split(".")[0])
        final_item.append(item[1])
        final_item.append(item[2])
        final_item.append(item[0])
        final_item.append(item[5])
        final_item.append(item[6])
        dates[item[6]] = 0
        final_item.append(item[10])
        final_item.append(item[9])
        final_item.append(item[4])
        final_list.append(final_item)
        ids[item[3].split(".")[0]] = 0


In [5]:
print(f"The original file has {len(chile)} data points. Without duplicates there are:{len(ids)}")
print(f"The earliest date is {min(dates.keys())} and the nearest date is {max(dates.keys())}")

The original file has 3248 data points. Without duplicates there are:860
The earliest date is 2011-01-04 and the nearest date is 2021-07-01


In [157]:
# Checking the differences with bucket files

language = "spanish"
in_prefix = f"{language}_documents/HSSC/sentences/"
ids_S3 = {}
for i, obj in enumerate(s3.Bucket(s3BucketName).objects.all().filter(Prefix = in_prefix)):
    if ".json" in obj.key and "ing" not in obj.key:
        name = obj.key.split("/")[-1].split("_")[0]
        ids_S3[name] = 0

In [158]:
print(f"The number of scraped items that are not in sentences is {len(ids.keys() - ids_S3.keys())}")

The number of scraped items that are not in sentences is 4


In [167]:
# Here we go and fetch the missing files and put them in the raw_pdf_updated folder
prefix = f"{language}_documents/text_files/HSSC/new/"

counter = 0
for item in final_list:
    if item[0] not in ids_S3:
        counter += 1
        key = prefix + item[0] + ".txt"
        with requests.get(item[8], stream=True) as r:
            s3.Object(s3BucketName, key).put(Body=r.content)

print(counter)

4


### Mexico

In [3]:
onlyfiles

['ElSalvador_20210730.csv',
 'USFR_2_20210311.csv',
 'Mexico_20210423.csv',
 'USFR_20210311.csv',
 'USFR_1_20210311.csv',
 'USFR_20210702.csv',
 'LeyChile_20210702.csv',
 'India_20210310.csv',
 'Mexico_20210703.csv',
 'USFR_20210310.csv',
 'India_20210703.csv',
 'India_20210812_2011.csv',
 'Mexico_20210809.csv',
 'El_Salvador_no_duplicates.csv']

In [37]:
# Data loading
file_position = 8
with open(path + onlyfiles[file_position], 'r', encoding = "Latin1") as f:#, encoding="ISO-8859-1"
    reader = csv.reader(f)
    mexico_july = list(reader)
    
file_position = 12
with open(path + onlyfiles[file_position], 'r', encoding = "Latin1") as f:#, encoding="ISO-8859-1"
    reader = csv.reader(f)
    mexico_august = list(reader)

id_july = [x[3].split(".")[0] for x in mexico_july]
id_august = [x[3].split(".")[0] for x in mexico_august]

code_july = [x[7] for x in mexico_july]
code_august = [x[7] for x in mexico_august]


In [38]:
print(f"July {len(id_july)} whithout dups {len(set(id_july))} - August {len(id_august)} whithout dups {len(set(id_august))}")
print(f"There are {len(set(id_july) - set(id_august))} files in the july dataset which are not in the august one")
print(f"There are {len(set(code_july) - set(code_august))} files by code in the july dataset which are not in the august one")

July 5092 whithout dups 5088 - August 5099 whithout dups 5099
There are 3973 files in the july dataset which are not in the august one
There are 0 files by code in the july dataset which are not in the august one


In [39]:
# Build the conversion dictionary from old codes to new ones

july_conv = dict((key[7], key[3].split(".")[0]) for key in mexico_july)
august_conv = dict((key[7], key[3].split(".")[0]) for key in mexico_august)

conversor = {}
for code, ID in july_conv.items():
    conversor[ID] = august_conv[code]

In [41]:
july_conv["c44f26663cde7e43afb625a4678de0b5f443a3ce"]

KeyError: 'c44f26663cde7e43afb625a4678de0b5f443a3ce'

In [42]:
august_conv["c44f26663cde7e43afb625a4678de0b5f443a3ce"]

KeyError: 'c44f26663cde7e43afb625a4678de0b5f443a3ce'

In [35]:
conversor

{'...',
 ...}

In [181]:
file_position = 6
with open(path + onlyfiles[file_position], 'r', encoding = "Latin1") as f:#, encoding="ISO-8859-1"
    reader = csv.reader(f)
    mexico = list(reader)
    
final_list = []
ids = {}
dates = {}
for item in mexico:
    if item[3].split(".")[0] not in ids:
        final_item = []
        final_item.append(item[3].split(".")[0])
        final_item.append(item[1])
        final_item.append(item[2])
        final_item.append(item[0])
        final_item.append(item[5])
        final_item.append(item[6])
        dates[item[6]] = 0
        final_item.append(item[10])
        final_item.append(item[9])
        final_item.append(item[4])
        final_list.append(final_item)
        ids[item[3].split(".")[0]] = 0


In [183]:
print(f"The original file has {len(mexico)} data points. Without duplicates there are:{len(ids)}")
print(f"The earliest date is {min(dates.keys())} and the nearest date is {max(dates.keys())}")

The original file has 5096 data points. Without duplicates there are:5096
The earliest date is 01-02-2011 and the nearest date is 31-12-2020


In [182]:
final_list[0]

['714697e02cb0b3f16cff58a1207334d942793cf2',
 'Mexico',
 'Diario Oficial de la Federacion',
 'PODER EJECUTIVO/SECRETARIA DE AGRICULTURA, GANADERIA, DESARROLLO RURAL, PESCA Y ALIMENTACION',
 '',
 '04-01-2011',
 'Aviso por el que se da a conocer información relativa a solicitudes de títulos de obtentor de variedades vegetales, correspondiente al mes de noviembre de 2010',
 '',
 'https://www.dof.gob.mx/nota_detalle.php?codigo=5173707&fecha=26/01/2011&print=true']

In [171]:
chile[0]

['MINISTERIO DEL MEDIO AMBIENTE; SERVICIO DE EVALUACIÓN AMBIENTAL; REGIÓN DE ÑUBLE',
 'Chile',
 'LeyChile',
 'c3df19a5008129ee6de996e8027cb0eacff09b4b.txt',
 'https://nuevo.leychile.cl/servicios/Consulta/Exportar?radioExportar=Normas&exportar_formato=txt&nombrearchivo=CHL/policy_1162116&exportar_con_notas_bcn=False&exportar_con_notas_originales=False&exportar_con_notas_al_pie=False&hddResultadoExportar=1162116.2021-07-01.0.0%23',
 'Resolución 96 EXENTA',
 '2021-07-01',
 'Resolución',
 'Federal',
 'Resolución',
 'NOTIFICA RESOLUCIÓN DE INICIO DE PROCESO DE PARTICIPACIÓN CIUDADANA EN DECLARACIÓN DE IMPACTO AMBIENTAL PROYECTO: "PLANTA FOTOVOLTAICA AGROVISIÓN"',
 'https://www.bcn.cl/leychile/navegar?idNorma=1162116']

In [2]:
path = "/home/propietari/Documents/fitxers importants/WRI/Scraping_results/March2021/"

onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]


In [3]:
onlyfiles

['Scraped_Documents_Chile.csv',
 'Oregon_20210218.csv',
 'El_Salvador_no_duplicates.tsv',
 'USFR_2_20210311.csv',
 'Scraped_Documents_ElSalvador.csv',
 'Mexico_20210423.csv',
 'Scraped_Documents_ElSalvador_all_filename.csv',
 'USFR_20210311.csv',
 'Scraped_Documents_ElSalvador_filename.csv',
 'USFR_1_20210311.csv',
 'LeyChile_20210215.csv',
 'India_20210310.csv',
 'ElSalvador_20210215.csv',
 'USFR_20210310.csv',
 'Scraped_Documents_ElSalvador_agricola_energia.csv',
 'El_Salvador_no_duplicates.csv']

In [13]:
file = path + "Scraped_Documents_Chile.csv"
with open(file, 'r', encoding = "Latin1") as f:
    reader = csv.reader(f)
    Chile = [row[13] for row in reader]

In [14]:
Chile[1]

'1150377'

In [15]:
print(f"There are {len(Chile)} entries and {len(set(Chile))} entries")

There are 4850 entries and 4850 entries


### India

In [39]:
# Loading the csv files from scraping to be compared
file_position = 7
with open(path + onlyfiles[file_position], 'r', encoding = "Latin1") as f:
    reader = csv.reader(f)
    india_march = [row[3] for row in reader]

file_position = 11
with open(path + onlyfiles[file_position], 'r', encoding = "Latin1") as f:
    reader = csv.reader(f)
    india_august = [row[7] for row in reader]

In [40]:
print(f"There are {len(india_march) - len(set(india_march))} duplicates in the march version")
print(f"There are {len(india_august) - len(set(india_august))} duplicates in the august version")
print(f"There are {len(set(india_march))} items in the March file and {len(set(india_august))} in the August one")
print(f"There are {len(set(india_march) - set(india_august))} files in the old version that are not in the new version")
print(f"There are {len(set(india_august) - set(india_march))} files in the new version that are not in the old version")
print(f"There are {len(set(india_march) | set(india_august))} unique ids in the union of the files")

There are 914 duplicates in the march version
There are 90 duplicates in the august version
There are 1040 items in the March file and 80 in the August one
There are 1040 files in the old version that are not in the new version
There are 80 files in the new version that are not in the old version
There are 1120 unique ids in the union of the files


In [42]:
file_position = 7
with open(path + onlyfiles[file_position], 'r', encoding="ISO-8859-1") as f:#, encoding = "Latin1"
    reader = csv.reader(f)
    india_march = list(reader)

file_position = 11
with open(path + onlyfiles[file_position], 'r', encoding="ISO-8859-1") as f:
    reader = csv.reader(f)
    india_august = list(reader)

In [43]:
india_march[5]

['',
 'India',
 'India Code',
 '43fd1c3c2b65e8a586a34995feb553be7de5ed8d.pdf',
 'https://upload.indiacode.nic.in/showfile?actid=AC_CEN_23_31_00002_200153_1517807322692&type=rule&filename=rules_2003_sw.pdf',
 'Rule',
 '12-09-2003',
 '',
 'Federal',
 '',
 'Protection of Plant Vhrieties and Farmersâ\x80\x99 Rights Rules, 2003.',
 'https://www.indiacode.nic.in/handle/123456789/4934?view_type=search&sam_handle=123456789/2513']

In [44]:
india_august[5]

['',
 'India',
 'India Code',
 '932ecb2a8473ee39cbae2b32e08b87707cabeb20.pdf',
 'https://upload.indiacode.nic.in/showfile?actid=AC_OR_80_611_00008_00008_1546412293540&type=rule&filename=orissa_non-gazetted_veterinary_technical_service_rules%2C_1983_-_f%26ard_department_notification_dt.4.1.1984.pdf',
 'Rule',
 '04-01-1984',
 '',
 ' Odisha',
 '',
 'Orissa Non-Gazetted Veterinary Technical Service Rules, 1983',
 'https://www.indiacode.nic.in/handle/123456789/5337?view_type=search&sam_handle=123456789/2502']

In [45]:
# Going to merge the two files in one. The fields were not saved in the same order so it is necessary to rearrange them

file_position = 7
with open(path + onlyfiles[file_position], 'r', encoding = "Latin1") as f:
    reader = csv.reader(f)
    india_march = list(reader)

file_position = 11
with open(path + onlyfiles[file_position], 'r') as f:
    reader = csv.reader(f)
    india_august = list(reader)
    
final_list = []
ids = {}
for item in india_march:
    if item[7] not in ids:
        final_item = []
        final_item.append(item[3].split(".")[0])
        final_item.append(item[1])
        final_item.append(item[2])
        final_item.append(item[0])
        final_item.append(item[5])
        final_item.append(item[6])
        final_item.append(item[10])
        final_item.append(item[9])
        final_item.append(item[11])
        final_list.append(final_item)
        ids[item[7]] = 0
    
for item in india_august:
    if item[3] not in ids:
        final_item = []
        final_item.append(item[3].split(".")[0])
        final_item.append(item[1])
        final_item.append(item[2])
        final_item.append(item[0])
        final_item.append(item[5])
        final_item.append(item[6])
        final_item.append(item[10])
        final_item.append(item[9])
        final_item.append(item[11])
        final_list.append(final_item)
        ids[item[3]] = 0

In [46]:
# Checking the differences with bucket files

language = "english"
in_prefix = f"{language}_documents/HSSC/sentences/"
ids_S3 = {}
for i, obj in enumerate(s3.Bucket(s3BucketName).objects.all().filter(Prefix = in_prefix)):
    if ".json" in obj.key and "ing" not in obj.key:
        name = obj.key.split("/")[-1].split("_")[0]
        ids_S3[name] = 0

In [47]:
print(f"The number of scraped items that are not in sentences is {len(ids.keys() - ids_S3.keys())}")

The number of scraped items that are not in sentences is 590
